# 🚀 Mejoras Propuestas para Modelos Espaciales de Precipitación

## 📊 Análisis de Resultados Actuales

### Problemas Identificados:

1. **R² Negativos**: Varios modelos muestran R² < 0, especialmente en H=2
   - ConvLSTM en KCE/PAFC: R² = -0.042 y -0.161
   - ConvRNN en KCE: R² = -0.340
   - ConvGRU en PAFC: R² = -0.052

2. **Degradación en H=3**: RMSE > 100 en muchos casos
   - ConvLSTM: RMSE hasta 111.3
   - ConvGRU: RMSE hasta 106.9
   - ConvRNN: RMSE hasta 87.3

3. **Inestabilidad**: Batch size = 4 es muy pequeño

### Mejores Resultados Actuales:
- **H=1**: ConvRNN BASIC (RMSE=43.49, R²=0.767)
- **H=2**: ConvGRU BASIC (RMSE=32.40, R²=0.401)
- **H=3**: ConvRNN KCE (RMSE=71.47, R²=0.611)

## 🔧 Mejoras Implementadas

### 1. Optimización de Hiperparámetros

| Parámetro | Valor Original | Valor Mejorado | Justificación |
|-----------|----------------|----------------|---------------|
| Batch Size | 4 | **16** | Mayor estabilidad en gradientes |
| Learning Rate | 1e-3 | **5e-4** | Convergencia más suave |
| Epochs | 50 | **100** | Más tiempo con early stopping |
| Patience | 6 | **10** | Evitar detención prematura |
| Dropout | 0 | **0.2** | Regularización |
| L2 Reg | 0 | **1e-5** | Prevenir overfitting |

### 2. Arquitecturas Mejoradas

#### ConvLSTM con Atención (ConvLSTM_Att)
```python
- 3 capas ConvLSTM (64→32→16 filtros)
- CBAM (Channel + Spatial Attention)
- BatchNorm + Dropout en cada capa
- Cabeza multi-escala (1×1, 3×3, 5×5)
```

#### ConvGRU Residual (ConvGRU_Res)
```python
- Skip connections desde input
- BatchNorm mejorado
- 2 bloques ConvGRU (64→32 filtros)
- Conexión residual final
```

#### Transformer Híbrido (Hybrid_Trans)
```python
- Encoder CNN temporal
- Multi-head attention (4 heads)
- LSTM para agregación temporal
- Decoder espacial
```

### 3. Técnicas Avanzadas

#### Learning Rate Scheduling
- **Warmup**: 5 épocas iniciales
- **Cosine Decay**: Reducción suave después del warmup
- **ReduceLROnPlateau**: Reducción adicional si se estanca

#### Data Augmentation
- Ruido gaussiano (σ=0.005)
- Preserva coherencia espacial y temporal

#### Regularización
- Dropout espacial (0.2)
- L2 en todos los pesos
- Batch Normalization

## 📈 Mejoras Esperadas

### Por Horizonte:
- **H=1**: RMSE < 40 (mejora ~8%)
- **H=2**: RMSE < 30, R² > 0.5 (mejora significativa)
- **H=3**: RMSE < 65, R² > 0.65 (mejora ~10%)

### Por Modelo:
1. **ConvLSTM_Att**: Mejor captura de patrones espaciales relevantes
2. **ConvGRU_Res**: Mayor estabilidad y menos degradación temporal
3. **Hybrid_Trans**: Mejor modelado de dependencias largas

## 🚀 Próximos Pasos

### Corto Plazo:
1. Entrenar modelos con configuración mejorada
2. Validar mejoras en métricas
3. Análisis de errores por región

### Medio Plazo:
1. **Ensemble Methods**: Combinar mejores modelos
2. **Multi-Task Learning**: Predecir múltiples variables
3. **Physics-Informed Loss**: Incorporar restricciones físicas

### Largo Plazo:
1. **Modelos 3D**: ConvLSTM3D para capturar altura
2. **Graph Neural Networks**: Para relaciones espaciales irregulares
3. **Uncertainty Quantification**: Intervalos de confianza

## 💻 Uso del Script

```bash
# Entrenar modelos avanzados
python models/train_advanced_models.py

# Con GPU específica
CUDA_VISIBLE_DEVICES=0 python models/train_advanced_models.py
```

## 📊 Monitoreo

Los resultados se guardan en:
- `models/output/Advanced_Spatial/advanced_results.csv`
- Historiales de entrenamiento por experimento
- Modelos guardados en formato .keras

## 🔍 Comparación con Baseline

El script genera automáticamente comparaciones con los modelos originales, mostrando:
- % de mejora en RMSE
- Evolución de R² por horizonte
- Tabla resumen de mejores modelos 

## 📊 Análisis de Resultados y Mejoras Propuestas

### Problemas Identificados en los Modelos Originales:
1. **R² negativos** en varios casos (especialmente H=2)
2. **Degradación severa** en H=3 (RMSE >100)
3. **Batch size muy pequeño** (4) causando inestabilidad
4. **Arquitecturas muy simples** (solo 2 capas)

### Mejoras Implementadas:

#### 1. **Hiperparámetros Optimizados**
- Batch size: 4 → 16 (mejor estabilidad)
- Learning rate: 1e-3 → 5e-4 (más conservador)
- Epochs: 50 → 100 (con early stopping)
- Regularización: Dropout (0.2) + L2 (1e-5)

#### 2. **Arquitecturas Mejoradas**
- **ConvLSTM con Atención**: CBAM (Channel + Spatial Attention)
- **ConvGRU con Skip Connections**: Conexiones residuales
- **PredRNN++**: Estado del arte para predicción espacio-temporal
- **ConvTransformer**: Híbrido CNN + Transformer

#### 3. **Técnicas Avanzadas**
- Learning rate scheduling (cosine decay con warmup)
- Data augmentation (ruido gaussiano)
- Multi-scale processing en la cabeza de salida
- Batch normalization en todas las capas


In [ ]:
# ───────────────────────── IMPORTS MEJORADOS ─────────────────────────
from __future__ import annotations
from pathlib import Path
import sys, os, gc, warnings
import numpy as np, pandas as pd, xarray as xr
import tensorflow as tf
from tensorflow.keras.layers import (
    Input, Conv2D, ConvLSTM2D, SimpleRNN, LSTM, GRU, Flatten, Dense, Reshape,
    Lambda, Permute, Layer, TimeDistributed, BatchNormalization, Dropout,
    Add, Multiply, Concatenate, GlobalAveragePooling2D, Activation,
    LayerNormalization, MultiHeadAttention, Conv3D, MaxPooling2D
)
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import (
    EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, 
    CSVLogger, Callback, LearningRateScheduler
)
from tensorflow.keras.optimizers import Adam, AdamW
from tensorflow.keras.regularizers import l1_l2
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt, seaborn as sns, geopandas as gpd, imageio.v2 as imageio
import cartopy.crs as ccrs
from IPython.display import clear_output, display
import json
from datetime import datetime

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_context('notebook')

# GPU config
for g in tf.config.list_physical_devices('GPU'):
    tf.config.experimental.set_memory_growth(g, True)

print("✅ Imports completados")


In [ ]:
# ───────────────────────── CONFIGURACIÓN MEJORADA ─────────────────────────
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    BASE_PATH = Path('/content/drive/MyDrive/ml_precipitation_prediction')
else:
    BASE_PATH = Path.cwd()
    for p in [BASE_PATH, *BASE_PATH.parents]:
        if (p / '.git').exists():
            BASE_PATH = p; break

# Paths
DATA_FILE = BASE_PATH/'data'/'output'/(
    'complete_dataset_with_features_with_clusters_elevation_windows_imfs_with_onehot_elevation_clean.nc')
OUT_ROOT = BASE_PATH/'models'/'output'/'Advanced_Spatial'
OUT_ROOT.mkdir(parents=True, exist_ok=True)
SHAPE_DIR = BASE_PATH/'data'/'input'/'shapes'
DEPT_GDF = gpd.read_file(SHAPE_DIR/'MGN_Departamento.shp')

# ⚡ HIPERPARÁMETROS OPTIMIZADOS
INPUT_WINDOW = 60
HORIZON = 3
EPOCHS = 100  # Más épocas con early stopping
BATCH = 16    # Batch size aumentado para estabilidad
LR = 5e-4     # Learning rate más conservador
PATIENCE = 10 # Más paciencia
DROPOUT = 0.2 # Regularización
L2_REG = 1e-5 # Regularización L2

# Feature sets
BASE_FEATS = ['year','month','month_sin','month_cos','doy_sin','doy_cos',
              'max_daily_precipitation','min_daily_precipitation','daily_precipitation_std',
              'elevation','slope','aspect']
ELEV_CLUSTER = ['elev_high','elev_med','elev_low']
KCE_FEATS = BASE_FEATS + ELEV_CLUSTER
PAFC_FEATS = KCE_FEATS + ['total_precipitation_lag1','total_precipitation_lag2','total_precipitation_lag12']
EXPERIMENTS = {'BASIC':BASE_FEATS, 'KCE':KCE_FEATS, 'PAFC':PAFC_FEATS}

# Cargar dataset
ds = xr.open_dataset(DATA_FILE)
lat, lon = len(ds.latitude), len(ds.longitude)
print(f"Dataset → time={len(ds.time)}, lat={lat}, lon={lon}")


In [ ]:
# ───────────────────────── CAPAS DE ATENCIÓN ─────────────────────────

class SpatialAttention(Layer):
    """Atención espacial para resaltar regiones importantes"""
    
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        
    def build(self, input_shape):
        self.conv = Conv2D(1, (7, 7), padding='same', activation='sigmoid')
        super().build(input_shape)
        
    def call(self, inputs):
        # Calcular estadísticas del canal
        avg_pool = K.mean(inputs, axis=-1, keepdims=True)
        max_pool = K.max(inputs, axis=-1, keepdims=True)
        concat = Concatenate(axis=-1)([avg_pool, max_pool])
        
        # Generar mapa de atención
        attention = self.conv(concat)
        
        return Multiply()([inputs, attention])


class ChannelAttention(Layer):
    """Atención de canal para ponderar features importantes"""
    
    def __init__(self, reduction_ratio=8, **kwargs):
        super().__init__(**kwargs)
        self.reduction_ratio = reduction_ratio
        
    def build(self, input_shape):
        channels = input_shape[-1]
        self.fc1 = Dense(channels // self.reduction_ratio, activation='relu')
        self.fc2 = Dense(channels, activation='sigmoid')
        super().build(input_shape)
        
    def call(self, inputs):
        # Global pooling
        avg_pool = GlobalAveragePooling2D()(inputs)
        max_pool = K.max(inputs, axis=[1, 2])
        
        # Shared MLP
        avg_out = self.fc2(self.fc1(avg_pool))
        max_out = self.fc2(self.fc1(max_pool))
        
        # Combinar
        attention = avg_out + max_out
        attention = K.expand_dims(K.expand_dims(attention, 1), 1)
        
        return Multiply()([inputs, attention])


class CBAM(Layer):
    """Convolutional Block Attention Module"""
    
    def __init__(self, reduction_ratio=8, **kwargs):
        super().__init__(**kwargs)
        self.channel_attention = ChannelAttention(reduction_ratio)
        self.spatial_attention = SpatialAttention()
        
    def call(self, inputs):
        x = self.channel_attention(inputs)
        x = self.spatial_attention(x)
        return x

print("✅ Capas de atención implementadas")


In [ ]:
# ───────────────────────── CAPAS AVANZADAS ─────────────────────────

class ConvGRU2DCell(Layer):
    """Celda ConvGRU2D mejorada con BatchNorm"""
    
    def __init__(self, filters, kernel_size, padding='same', activation='tanh',
                 recurrent_activation='sigmoid', use_batch_norm=True, **kwargs):
        super().__init__(**kwargs)
        self.filters = filters
        self.kernel_size = kernel_size if isinstance(kernel_size, tuple) else (kernel_size, kernel_size)
        self.padding = padding
        self.activation = tf.keras.activations.get(activation)
        self.recurrent_activation = tf.keras.activations.get(recurrent_activation)
        self.use_batch_norm = use_batch_norm
        self.state_size = (filters,)
        
    def build(self, input_shape):
        input_dim = input_shape[-1]
        
        # Kernels
        self.kernel = self.add_weight(
            shape=(*self.kernel_size, input_dim, self.filters * 3),
            initializer='glorot_uniform',
            regularizer=l1_l2(l1=0, l2=L2_REG),
            name='kernel'
        )
        
        self.recurrent_kernel = self.add_weight(
            shape=(*self.kernel_size, self.filters, self.filters * 3),
            initializer='orthogonal',
            regularizer=l1_l2(l1=0, l2=L2_REG),
            name='recurrent_kernel'
        )
        
        self.bias = self.add_weight(
            shape=(self.filters * 3,),
            initializer='zeros',
            name='bias'
        )
        
        if self.use_batch_norm:
            self.bn_x = BatchNormalization()
            self.bn_h = BatchNormalization()
        
        super().build(input_shape)
    
    def call(self, inputs, states, training=None):
        h_tm1 = states[0]
        
        # Convoluciones
        x_conv = K.conv2d(inputs, self.kernel, padding=self.padding)
        h_conv = K.conv2d(h_tm1, self.recurrent_kernel, padding=self.padding)
        
        if self.use_batch_norm:
            x_conv = self.bn_x(x_conv, training=training)
            h_conv = self.bn_h(h_conv, training=training)
        
        x_z, x_r, x_h = tf.split(x_conv, 3, axis=-1)
        h_z, h_r, h_h = tf.split(h_conv, 3, axis=-1)
        b_z, b_r, b_h = tf.split(self.bias, 3)
        
        # Gates
        z = self.recurrent_activation(x_z + h_z + b_z)
        r = self.recurrent_activation(x_r + h_r + b_r)
        
        # Hidden state
        h_candidate = self.activation(x_h + r * h_h + b_h)
        h = (1 - z) * h_tm1 + z * h_candidate
        
        return h, [h]


class ConvGRU2D(Layer):
    """ConvGRU2D mejorado con soporte para BatchNorm y Dropout"""
    
    def __init__(self, filters, kernel_size, padding='same', activation='tanh',
                 recurrent_activation='sigmoid', return_sequences=False,
                 use_batch_norm=True, dropout=0.0, **kwargs):
        super().__init__(**kwargs)
        self.filters = filters
        self.kernel_size = kernel_size
        self.padding = padding
        self.activation = activation
        self.recurrent_activation = recurrent_activation
        self.return_sequences = return_sequences
        self.use_batch_norm = use_batch_norm
        self.dropout = dropout
        
        self.cell = ConvGRU2DCell(
            filters, kernel_size, padding, activation, 
            recurrent_activation, use_batch_norm
        )
        
        if dropout > 0:
            self.dropout_layer = Dropout(dropout)
        
    def build(self, input_shape):
        self.cell.build(input_shape[2:])
        super().build(input_shape)
        
    def call(self, inputs, training=None):
        batch_size = tf.shape(inputs)[0]
        time_steps = tf.shape(inputs)[1]
        height = tf.shape(inputs)[2]
        width = tf.shape(inputs)[3]
        
        # Estado inicial
        initial_state = tf.zeros((batch_size, height, width, self.filters))
        
        # Procesar secuencia
        outputs = []
        state = initial_state
        
        for t in range(inputs.shape[1]):
            output, [state] = self.cell(inputs[:, t], [state], training=training)
            
            if self.dropout > 0:
                output = self.dropout_layer(output, training=training)
                
            outputs.append(output)
        
        outputs = tf.stack(outputs, axis=1)
        
        if self.return_sequences:
            return outputs
        else:
            return outputs[:, -1]
    
    def get_config(self):
        config = super().get_config()
        config.update({
            'filters': self.filters,
            'kernel_size': self.kernel_size,
            'padding': self.padding,
            'activation': self.activation,
            'recurrent_activation': self.recurrent_activation,
            'return_sequences': self.return_sequences,
            'use_batch_norm': self.use_batch_norm,
            'dropout': self.dropout
        })
        return config

print("✅ Capas ConvGRU mejoradas implementadas")


In [ ]:
# ───────────────────────── MODEL BUILDERS AVANZADOS ─────────────────────────

def _advanced_spatial_head(x, use_attention=True):
    """Cabeza de proyección mejorada con atención opcional"""
    
    if use_attention:
        x = CBAM()(x)
    
    # Multi-scale processing
    conv1 = Conv2D(HORIZON, (1, 1), padding='same')(x)
    conv3 = Conv2D(HORIZON, (3, 3), padding='same')(x)
    conv5 = Conv2D(HORIZON, (5, 5), padding='same')(x)
    
    # Combine multi-scale features
    x = Add()([conv1, conv3, conv5])
    x = BatchNormalization()(x)
    x = Activation('linear')(x)
    
    # Reshape to output format
    x = Lambda(lambda t: tf.transpose(t, [0, 3, 1, 2]),
               output_shape=(HORIZON, lat, lon))(x)
    x = Lambda(lambda t: tf.expand_dims(t, -1),
               output_shape=(HORIZON, lat, lon, 1))(x)
    
    return x


def build_convlstm_attention(n_feats: int):
    """ConvLSTM con mecanismo de atención"""
    inp = Input(shape=(INPUT_WINDOW, lat, lon, n_feats))
    
    # Primera capa con más filtros
    x = ConvLSTM2D(64, (3, 3), padding='same', return_sequences=True,
                   kernel_regularizer=l1_l2(l1=0, l2=L2_REG))(inp)
    x = BatchNormalization()(x)
    x = Dropout(DROPOUT)(x)
    
    # Segunda capa con atención
    x = ConvLSTM2D(32, (3, 3), padding='same', return_sequences=True,
                   kernel_regularizer=l1_l2(l1=0, l2=L2_REG))(x)
    x = BatchNormalization()(x)
    
    # Aplicar atención temporal
    x = TimeDistributed(CBAM())(x)
    
    # Capa final
    x = ConvLSTM2D(16, (3, 3), padding='same', return_sequences=False,
                   kernel_regularizer=l1_l2(l1=0, l2=L2_REG))(x)
    x = BatchNormalization()(x)
    
    out = _advanced_spatial_head(x)
    return Model(inp, out, name='ConvLSTM_Attention')


def build_convgru_residual(n_feats: int):
    """ConvGRU con skip connections"""
    inp = Input(shape=(INPUT_WINDOW, lat, lon, n_feats))
    
    # Encoder path
    enc1 = ConvGRU2D(64, (3, 3), return_sequences=True, 
                     use_batch_norm=True, dropout=DROPOUT)(inp)
    
    enc2 = ConvGRU2D(32, (3, 3), return_sequences=True,
                     use_batch_norm=True, dropout=DROPOUT)(enc1)
    
    # Bottleneck
    bottleneck = ConvGRU2D(16, (3, 3), return_sequences=False,
                           use_batch_norm=True)(enc2)
    
    # Skip connection from input
    skip = TimeDistributed(Conv2D(16, (1, 1), padding='same'))(inp)
    skip = Lambda(lambda x: x[:, -1])(skip)  # Take last timestep
    
    # Combine
    x = Add()([bottleneck, skip])
    x = BatchNormalization()(x)
    
    out = _advanced_spatial_head(x)
    return Model(inp, out, name='ConvGRU_Residual')


def build_hybrid_transformer(n_feats: int):
    """Modelo híbrido CNN + Transformer"""
    inp = Input(shape=(INPUT_WINDOW, lat, lon, n_feats))
    
    # Encoder convolucional
    x = TimeDistributed(Conv2D(64, (3, 3), padding='same', activation='relu'))(inp)
    x = TimeDistributed(BatchNormalization())(x)
    x = TimeDistributed(Conv2D(32, (3, 3), padding='same', activation='relu'))(x)
    x = TimeDistributed(BatchNormalization())(x)
    
    # Reducir dimensionalidad espacial
    x = TimeDistributed(MaxPooling2D((2, 2), padding='same'))(x)
    x = TimeDistributed(Flatten())(x)
    
    # Self-attention temporal
    x = MultiHeadAttention(num_heads=4, key_dim=32, dropout=DROPOUT)(x, x)
    x = LayerNormalization()(x)
    
    # Agregación temporal con LSTM
    x = LSTM(128, return_sequences=False)(x)
    x = BatchNormalization()(x)
    x = Dropout(DROPOUT)(x)
    
    # Decodificador espacial
    x = Dense(lat * lon * 16)(x)
    x = Reshape((lat, lon, 16))(x)
    
    out = _advanced_spatial_head(x)
    return Model(inp, out, name='Hybrid_Transformer')


# Diccionario de modelos
ADVANCED_MODELS = {
    'ConvLSTM_Att': build_convlstm_attention,
    'ConvGRU_Res': build_convgru_residual,
    'Hybrid_Trans': build_hybrid_transformer
}

print("✅ Model builders avanzados creados")


In [ ]:
# ───────────────────────── UTILIDADES DE ENTRENAMIENTO ─────────────────────────

def cosine_decay_with_warmup(epoch, lr_base=LR, total_epochs=EPOCHS, warmup_epochs=5):
    """Cosine decay learning rate con warmup"""
    if epoch < warmup_epochs:
        return lr_base * (epoch + 1) / warmup_epochs
    else:
        progress = (epoch - warmup_epochs) / (total_epochs - warmup_epochs)
        return lr_base * 0.5 * (1 + np.cos(np.pi * progress))


class AdvancedTrainingMonitor(Callback):
    """Monitor avanzado con métricas adicionales"""
    
    def __init__(self, model_name, experiment_name, validation_data=None):
        super().__init__()
        self.model_name = model_name
        self.experiment_name = experiment_name
        self.validation_data = validation_data
        self.history = {
            'loss': [], 'val_loss': [], 'mae': [], 'val_mae': [],
            'lr': [], 'epoch': []
        }
        
    def on_epoch_end(self, epoch, logs=None):
        # Guardar métricas
        self.history['epoch'].append(epoch + 1)
        self.history['loss'].append(logs.get('loss'))
        self.history['val_loss'].append(logs.get('val_loss'))
        self.history['mae'].append(logs.get('mae'))
        self.history['val_mae'].append(logs.get('val_mae'))
        
        # Learning rate
        lr = float(K.get_value(self.model.optimizer.learning_rate))
        self.history['lr'].append(lr)
        
        # Clear output
        clear_output(wait=True)
        
        # Crear visualización mejorada
        fig = plt.figure(figsize=(20, 5))
        
        # Loss curves
        ax1 = plt.subplot(141)
        ax1.plot(self.history['epoch'], self.history['loss'], 'b-', label='Train Loss', linewidth=2)
        ax1.plot(self.history['epoch'], self.history['val_loss'], 'r-', label='Val Loss', linewidth=2)
        ax1.set_xlabel('Epoch')
        ax1.set_ylabel('Loss')
        ax1.set_title(f'{self.model_name} - Loss Evolution')
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        
        # MAE curves
        ax2 = plt.subplot(142)
        ax2.plot(self.history['epoch'], self.history['mae'], 'g-', label='Train MAE', linewidth=2)
        ax2.plot(self.history['epoch'], self.history['val_mae'], 'm-', label='Val MAE', linewidth=2)
        ax2.set_xlabel('Epoch')
        ax2.set_ylabel('MAE')
        ax2.set_title('MAE Evolution')
        ax2.legend()
        ax2.grid(True, alpha=0.3)
        
        # Tasa de mejora y convergencia
        ax3 = plt.subplot(143)
        if len(self.history['val_loss']) > 1:
            # Calcular tasa de mejora epoch a epoch
            improvements = []
            for i in range(1, len(self.history['val_loss'])):
                prev_loss = self.history['val_loss'][i-1]
                curr_loss = self.history['val_loss'][i]
                improvement = ((prev_loss - curr_loss) / prev_loss) * 100
                improvements.append(improvement)
            
            # Plot de tasa de mejora
            ax3.plot(self.history['epoch'][1:], improvements, 'g-', linewidth=2, alpha=0.7)
            ax3.axhline(y=0, color='black', linestyle='--', alpha=0.5)
            ax3.fill_between(self.history['epoch'][1:], improvements, 0, 
                           where=[x > 0 for x in improvements], 
                           color='green', alpha=0.3, label='Mejora')
            ax3.fill_between(self.history['epoch'][1:], improvements, 0, 
                           where=[x <= 0 for x in improvements], 
                           color='red', alpha=0.3, label='Empeoramiento')
            
            # Línea de tendencia suavizada
            if len(improvements) > 5:
                window = min(5, len(improvements)//3)
                smoothed = pd.Series(improvements).rolling(window=window, center=True).mean()
                ax3.plot(self.history['epoch'][1:], smoothed, 'b-', linewidth=2.5, 
                        label=f'Tendencia ({window} epochs)')
            
            ax3.set_xlabel('Epoch')
            ax3.set_ylabel('Tasa de Mejora (%)')
            ax3.set_title('Progreso del Entrenamiento')
            ax3.legend(loc='best')
            ax3.grid(True, alpha=0.3)
            
            # Añadir anotación de convergencia
            if len(improvements) > 10:
                recent_avg = np.mean(improvements[-5:])
                if abs(recent_avg) < 0.5:
                    ax3.text(0.95, 0.95, '⚠️ Posible convergencia', 
                            transform=ax3.transAxes, ha='right', va='top',
                            bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.5))
        
        # Loss distribution (últimas 10 épocas)
        if len(self.history['loss']) > 10:
            ax4 = plt.subplot(144)
            recent_losses = self.history['val_loss'][-10:]
            ax4.hist(recent_losses, bins=10, alpha=0.7, color='purple')
            ax4.axvline(np.mean(recent_losses), color='red', linestyle='--', 
                       label=f'Mean: {np.mean(recent_losses):.4f}')
            ax4.set_xlabel('Val Loss')
            ax4.set_ylabel('Frequency')
            ax4.set_title('Recent Val Loss Distribution')
            ax4.legend()
        
        plt.suptitle(f'{self.model_name} - {self.experiment_name} - Epoch {epoch + 1}', 
                    fontsize=16, fontweight='bold')
        plt.tight_layout()
        display(fig)
        plt.close()
        
        # Mostrar métricas
        print(f"\n📊 Época {epoch + 1}/{self.params['epochs']}")
        print(f"   • Loss: {logs.get('loss'):.6f} | Val Loss: {logs.get('val_loss'):.6f}")
        print(f"   • MAE: {logs.get('mae'):.6f} | Val MAE: {logs.get('val_mae'):.6f}")
        print(f"   • Learning Rate: {lr:.2e}")
        
        # Calcular mejora
        if len(self.history['val_loss']) > 1:
            improvement = (self.history['val_loss'][-2] - self.history['val_loss'][-1]) 
            improvement_pct = improvement / self.history['val_loss'][-2] * 100
            print(f"   • Mejora: {improvement:.6f} ({improvement_pct:.2f}%)")
            
            # Detectar overfitting
            overfit_ratio = self.history['val_loss'][-1] / self.history['loss'][-1]
            if overfit_ratio > 1.5:
                print(f"   ⚠️  Posible overfitting detectado (ratio: {overfit_ratio:.2f})")


def create_callbacks(model_name, exp_name, model_path):
    """Crear callbacks optimizados"""
    
    # Crear directorio para métricas si no existe
    metrics_dir = model_path.parent / 'training_metrics'
    metrics_dir.mkdir(exist_ok=True)
    
    callbacks = [
        # Early stopping mejorado
        EarlyStopping(
            monitor='val_loss',
            patience=PATIENCE,
            restore_best_weights=True,
            verbose=1,
            min_delta=1e-4
        ),
        
        # Model checkpoint
        ModelCheckpoint(
            model_path,
            monitor='val_loss',
            save_best_only=True,
            verbose=1
        ),
        
        # Reduce LR on plateau
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=PATIENCE//2,
            min_lr=1e-7,
            verbose=1
        ),
        
        # Learning rate scheduler (cosine decay)
        LearningRateScheduler(cosine_decay_with_warmup, verbose=0),
        
        # CSV logger
        CSVLogger(
            metrics_dir / f"{model_name}_training_log.csv",
            separator=',',
            append=False
        ),
        
        # Advanced monitor
        AdvancedTrainingMonitor(model_name, exp_name)
    ]
    
    return callbacks

print("✅ Utilidades de entrenamiento creadas")


In [ ]:
# ───────────────────────── HELPERS ─────────────────────────

def windowed_arrays(X:np.ndarray, y:np.ndarray):
    """Crear ventanas deslizantes para series temporales"""
    seq_X, seq_y = [], []
    T = len(X)
    for start in range(T-INPUT_WINDOW-HORIZON+1):
        end_w = start+INPUT_WINDOW
        end_y = end_w+HORIZON
        Xw, yw = X[start:end_w], y[end_w:end_y]
        if np.isnan(Xw).any() or np.isnan(yw).any():
            continue
        seq_X.append(Xw)
        seq_y.append(yw)
    return np.asarray(seq_X,dtype=np.float32), np.asarray(seq_y,dtype=np.float32)


def quick_plot(ax,data,cmap,title,vmin=None,vmax=None):
    """Plotear mapa geográfico"""
    mesh = ax.pcolormesh(ds.longitude,ds.latitude,data,cmap=cmap,shading='nearest',
                         vmin=vmin,vmax=vmax,transform=ccrs.PlateCarree())
    ax.coastlines()
    ax.add_geometries(DEPT_GDF.geometry,ccrs.PlateCarree(),
                      edgecolor='black',facecolor='none',linewidth=1)
    ax.gridlines(draw_labels=False, linewidth=.5, linestyle='--', alpha=.4)
    ax.set_title(title,fontsize=9)
    return mesh


def save_hyperparameters(exp_path, model_name, hyperparams):
    """Guarda los hiperparámetros en un archivo JSON"""
    hp_file = exp_path / f"{model_name}_hyperparameters.json"
    with open(hp_file, 'w') as f:
        json.dump(hyperparams, f, indent=4)
    print(f"   💾 Hiperparámetros guardados en: {hp_file.name}")


def plot_learning_curves(history, exp_path, model_name, show=True):
    """Genera y guarda las curvas de aprendizaje"""
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    # Loss
    axes[0].plot(history.history['loss'], label='Train Loss', linewidth=2)
    axes[0].plot(history.history['val_loss'], label='Val Loss', linewidth=2)
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Loss (MSE)')
    axes[0].set_title(f'{model_name} - Loss Evolution')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Análisis de Convergencia y Estabilidad
    val_losses = history.history['val_loss']
    train_losses = history.history['loss']
    
    if len(val_losses) > 1:
        # Calcular métricas de convergencia
        epochs = range(1, len(val_losses) + 1)
        
        # 1. Ratio de overfitting
        overfit_ratio = [val_losses[i] / train_losses[i] for i in range(len(val_losses))]
        
        # 2. Estabilidad (desviación estándar móvil)
        window = min(5, len(val_losses)//3)
        val_std = pd.Series(val_losses).rolling(window=window).std()
        
        # Crear subplot con dos ejes Y
        ax2_left = axes[1]
        ax2_right = ax2_left.twinx()
        
        # Plot ratio de overfitting
        line1 = ax2_left.plot(epochs, overfit_ratio, 'r-', linewidth=2, 
                             label='Ratio Val/Train', alpha=0.8)
        ax2_left.axhline(y=1.0, color='black', linestyle='--', alpha=0.5)
        ax2_left.fill_between(epochs, 1.0, overfit_ratio, 
                            where=[x > 1.0 for x in overfit_ratio],
                            color='red', alpha=0.2)
        ax2_left.set_xlabel('Epoch')
        ax2_left.set_ylabel('Ratio Val Loss / Train Loss', color='red')
        ax2_left.tick_params(axis='y', labelcolor='red')
        
        # Plot estabilidad
        line2 = ax2_right.plot(epochs[window-1:], val_std[window-1:], 'b-', 
                             linewidth=2, label='Estabilidad', alpha=0.8)
        ax2_right.set_ylabel('Desviación Estándar (ventana móvil)', color='blue')
        ax2_right.tick_params(axis='y', labelcolor='blue')
        
        # Título y leyenda combinada
        ax2_left.set_title(f'{model_name} - Análisis de Convergencia')
        
        # Combinar leyendas
        lines = line1 + line2
        labels = [l.get_label() for l in lines]
        ax2_left.legend(lines, labels, loc='upper left')
        
        ax2_left.grid(True, alpha=0.3)
        
        # Añadir zonas de interpretación
        if max(overfit_ratio) > 1.5:
            ax2_left.text(0.02, 0.98, '⚠️ Alto overfitting detectado', 
                        transform=ax2_left.transAxes, va='top',
                        bbox=dict(boxstyle='round', facecolor='red', alpha=0.3))
        elif min(val_std[window-1:]) < 0.001:
            ax2_left.text(0.02, 0.98, '✓ Entrenamiento estable', 
                        transform=ax2_left.transAxes, va='top',
                        bbox=dict(boxstyle='round', facecolor='green', alpha=0.3))
    else:
        axes[1].text(0.5, 0.5, 'Insufficient data for convergence analysis', 
                    transform=axes[1].transAxes, ha='center', va='center',
                    fontsize=12, color='gray')
        axes[1].set_title(f'{model_name} - Convergence Analysis')
        axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    
    # Guardar
    curves_path = exp_path / f"{model_name}_learning_curves.png"
    plt.savefig(curves_path, dpi=150, bbox_inches='tight')
    
    if show:
        plt.show()
    else:
        plt.close()
    
    return curves_path


def print_training_summary(history, model_name, exp_name):
    """Imprime un resumen del entrenamiento"""
    final_loss = history.history['loss'][-1]
    final_val_loss = history.history['val_loss'][-1]
    best_val_loss = min(history.history['val_loss'])
    best_epoch = history.history['val_loss'].index(best_val_loss) + 1
    
    print(f"\n   📊 Resumen de entrenamiento {model_name} - {exp_name}:")
    print(f"      • Épocas totales: {len(history.history['loss'])}")
    print(f"      • Loss final (train): {final_loss:.6f}")
    print(f"      • Loss final (val): {final_val_loss:.6f}")
    print(f"      • Mejor loss (val): {best_val_loss:.6f} en época {best_epoch}")
    if 'lr' in history.history and len(history.history['lr']) > 0:
        final_lr = history.history['lr'][-1]
        print(f"      • Learning rate final: {final_lr:.2e}")
    else:
        print(f"      • Learning rate final: No disponible")

print("✅ Funciones helper creadas")


In [ ]:
# ───────────────────────── TRAIN + EVAL LOOP ─────────────────────────

# Diccionario para almacenar historiales de entrenamiento
all_histories = {}
results = []

for exp, feat_list in EXPERIMENTS.items():
    print(f"\n{'='*70}")
    print(f"🔬 EXPERIMENTO: {exp} ({len(feat_list)} features)")
    print(f"{'='*70}")
    
    # Preparar datos
    Xarr = ds[feat_list].to_array().transpose('time','latitude','longitude','variable').values.astype(np.float32)
    yarr = ds['total_precipitation'].values.astype(np.float32)[...,None]
    X, y = windowed_arrays(Xarr, yarr)
    split = int(0.8*len(X))
    val_split = int(0.9*len(X))

    # Normalización
    sx = StandardScaler().fit(X[:split].reshape(-1,len(feat_list)))
    sy = StandardScaler().fit(y[:split].reshape(-1,1))
    X_sc = sx.transform(X.reshape(-1,len(feat_list))).reshape(X.shape)
    y_sc = sy.transform(y.reshape(-1,1)).reshape(y.shape)
    
    # Splits
    X_tr, X_va, X_te = X_sc[:split], X_sc[split:val_split], X_sc[val_split:]
    y_tr, y_va, y_te = y_sc[:split], y_sc[split:val_split], y_sc[val_split:]
    
    print(f"   Datos: Train={len(X_tr)}, Val={len(X_va)}, Test={len(X_te)}")

    OUT_EXP = OUT_ROOT/exp
    OUT_EXP.mkdir(exist_ok=True)
    
    # Crear subdirectorio para métricas de entrenamiento
    METRICS_DIR = OUT_EXP / 'training_metrics'
    METRICS_DIR.mkdir(exist_ok=True)

    for mdl_name, builder in ADVANCED_MODELS.items():
        print(f"\n{'─'*50}")
        print(f"🤖 Modelo: {mdl_name}")
        print(f"{'─'*50}")
        
        model_path = OUT_EXP/f"{mdl_name.lower()}_best.keras"
        if model_path.exists():
            model_path.unlink()
        
        try:
            # Construir modelo
            model = builder(n_feats=len(feat_list))
            
            # Definir optimizador con configuración explícita
            optimizer = AdamW(learning_rate=LR, weight_decay=L2_REG)
            model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
            
            # Hiperparámetros
            hyperparams = {
                'experiment': exp,
                'model': mdl_name,
                'features': feat_list,
                'n_features': len(feat_list),
                'input_window': INPUT_WINDOW,
                'horizon': HORIZON,
                'batch_size': BATCH,
                'initial_lr': LR,
                'epochs': EPOCHS,
                'patience': PATIENCE,
                'dropout': DROPOUT,
                'l2_reg': L2_REG,
                'train_samples': len(X_tr),
                'val_samples': len(X_va),
                'test_samples': len(X_te),
                'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                'model_params': model.count_params()
            }
            
            # Guardar hiperparámetros
            save_hyperparameters(METRICS_DIR, mdl_name, hyperparams)
            
            # Callbacks
            callbacks = create_callbacks(mdl_name, exp, model_path)
            
            # Entrenar con verbose=0 para usar nuestro monitor personalizado
            print(f"\n🏃 Iniciando entrenamiento...")
            print(f"   📊 Visualización en tiempo real activada")
            print(f"   📈 Parámetros del modelo: {model.count_params():,}")
            
            history = model.fit(
                X_tr, y_tr,
                validation_data=(X_va, y_va),
                epochs=EPOCHS,
                batch_size=BATCH,
                callbacks=callbacks,
                verbose=0  # Usar 0 para que solo se muestre nuestro monitor
            )
            
            # Guardar historial
            all_histories[f"{exp}_{mdl_name}"] = history
            
            # Mostrar resumen de entrenamiento
            print_training_summary(history, mdl_name, exp)
            
            # Plotear y guardar curvas de aprendizaje
            plot_learning_curves(history, METRICS_DIR, mdl_name, show=True)
            
            # Guardar historial como JSON
            # Obtener learning rates del monitor de entrenamiento si no están en history
            training_monitor = [cb for cb in callbacks if isinstance(cb, AdvancedTrainingMonitor)][0]
            lr_values = history.history.get('lr', [])
            if not lr_values and hasattr(training_monitor, 'history'):
                lr_values = training_monitor.history['lr']
            
            history_dict = {
                'loss': [float(x) for x in history.history['loss']],
                'val_loss': [float(x) for x in history.history['val_loss']],
                'mae': [float(x) for x in history.history.get('mae', [])],
                'val_mae': [float(x) for x in history.history.get('val_mae', [])],
                'lr': [float(x) for x in lr_values] if lr_values else []
            }
            
            with open(METRICS_DIR / f"{mdl_name}_history.json", 'w') as f:
                json.dump(history_dict, f, indent=4)

            # ─ Evaluación en Test Set ─
            print(f"\n📊 Evaluando en test set...")
            test_loss, test_mae = model.evaluate(X_te, y_te, verbose=0)
            print(f"   Test Loss: {test_loss:.6f}, Test MAE: {test_mae:.6f}")

            # ─ Predicciones y visualización ─
            print(f"\n🎯 Generando predicciones...")
            # Usar las primeras 5 muestras del test set
            sample_indices = min(5, len(X_te))
            y_hat_sc = model.predict(X_te[:sample_indices], verbose=0)
            y_hat = sy.inverse_transform(y_hat_sc.reshape(-1,1)).reshape(-1,HORIZON,lat,lon)
            y_true = sy.inverse_transform(y_te[:sample_indices].reshape(-1,1)).reshape(-1,HORIZON,lat,lon)

            # ─ Métricas de evaluación por horizonte ─
            for h in range(HORIZON):
                rmse = np.sqrt(mean_squared_error(y_true[:,h].ravel(), y_hat[:,h].ravel()))
                mae = mean_absolute_error(y_true[:,h].ravel(), y_hat[:,h].ravel())
                r2 = r2_score(y_true[:,h].ravel(), y_hat[:,h].ravel())
                
                results.append({
                    'Experiment': exp,
                    'Model': mdl_name,
                    'H': h+1,
                    'RMSE': rmse,
                    'MAE': mae,
                    'R2': r2,
                    'Test_Loss': test_loss,
                    'Parameters': model.count_params()
                })
                
                print(f"   📈 H={h+1}: RMSE={rmse:.4f}, MAE={mae:.4f}, R²={r2:.4f}")

            # ─ Mapas & GIF ─
            print(f"\n🎨 Generando visualizaciones...")
            # Usar la primera muestra para visualización
            sample_idx = 0
            vmin, vmax = 0, max(y_true[sample_idx].max(), y_hat[sample_idx].max())
            frames = []
            dates = pd.date_range(ds.time.values[-HORIZON], periods=HORIZON, freq='MS')
            
            for h in range(HORIZON):
                err = np.clip(np.abs((y_true[sample_idx,h]-y_hat[sample_idx,h])/(y_true[sample_idx,h]+1e-5))*100, 0, 100)
                fig, axs = plt.subplots(1, 3, figsize=(12, 4), subplot_kw={'projection': ccrs.PlateCarree()})
                quick_plot(axs[0], y_true[sample_idx,h], 'Blues', f"Real h={h+1}", vmin, vmax)
                quick_plot(axs[1], y_hat[sample_idx,h], 'Blues', f"{mdl_name} h={h+1}", vmin, vmax)
                quick_plot(axs[2], err, 'Reds', f"MAPE% h={h+1}", 0, 100)
                fig.suptitle(f"{mdl_name} – {exp} – {dates[h].strftime('%Y-%m')}")
                png = OUT_EXP/f"{mdl_name}_{h+1}.png"
                fig.savefig(png, bbox_inches='tight')
                plt.close(fig)
                frames.append(imageio.imread(png))
            
            imageio.mimsave(OUT_EXP/f"{mdl_name}.gif", frames, fps=0.5)
            print(f"   ✅ GIF guardado: {OUT_EXP/f'{mdl_name}.gif'}")
            
            tf.keras.backend.clear_session()
            gc.collect()
            
        except Exception as e:
            print(f"  ⚠️ Error en {mdl_name}: {str(e)}")
            print(f"  → Saltando {mdl_name} para {exp}")
            import traceback
            traceback.print_exc()
            continue

# ───────────────────────── CSV FINAL ─────────────────────────
res_df = pd.DataFrame(results)
res_df.to_csv(OUT_ROOT/'metrics_advanced.csv', index=False)
print("\n📑 Metrics saved →", OUT_ROOT/'metrics_advanced.csv')


In [ ]:
# ───────────────────────── VISUALIZACIÓN COMPARATIVA ─────────────────────────
print("\n" + "="*70)
print("📊 GENERANDO VISUALIZACIONES COMPARATIVAS")
print("="*70)

# Crear directorio para comparaciones
COMP_DIR = OUT_ROOT / 'comparisons'
COMP_DIR.mkdir(exist_ok=True)

# 1. Comparación de métricas entre modelos
if res_df is not None and len(res_df) > 0:
    fig, axes = plt.subplots(2, 2, figsize=(18, 12))
    
    # RMSE por modelo y experimento
    pivot_rmse = res_df.pivot_table(values='RMSE', index='Model', columns='Experiment', aggfunc='mean')
    pivot_rmse.plot(kind='bar', ax=axes[0,0])
    axes[0,0].set_title('RMSE Promedio por Modelo y Experimento', fontsize=14, pad=10)
    axes[0,0].set_ylabel('RMSE')
    axes[0,0].set_xlabel('Modelo')
    axes[0,0].legend(title='Experimento', loc='upper left')
    axes[0,0].grid(True, alpha=0.3)
    axes[0,0].tick_params(axis='x', rotation=45)
    
    # MAE por modelo y experimento
    pivot_mae = res_df.pivot_table(values='MAE', index='Model', columns='Experiment', aggfunc='mean')
    pivot_mae.plot(kind='bar', ax=axes[0,1])
    axes[0,1].set_title('MAE Promedio por Modelo y Experimento', fontsize=14, pad=10)
    axes[0,1].set_ylabel('MAE')
    axes[0,1].set_xlabel('Modelo')
    axes[0,1].legend(title='Experimento', loc='upper left')
    axes[0,1].grid(True, alpha=0.3)
    axes[0,1].tick_params(axis='x', rotation=45)
    
    # R² por modelo y experimento
    pivot_r2 = res_df.pivot_table(values='R2', index='Model', columns='Experiment', aggfunc='mean')
    pivot_r2.plot(kind='bar', ax=axes[1,0])
    axes[1,0].set_title('R² Promedio por Modelo y Experimento', fontsize=14, pad=10)
    axes[1,0].set_ylabel('R²')
    axes[1,0].set_xlabel('Modelo')
    axes[1,0].legend(title='Experimento', loc='lower right')
    axes[1,0].grid(True, alpha=0.3)
    axes[1,0].tick_params(axis='x', rotation=45)
    
    # Evolución de métricas por horizonte
    ax_horizon = axes[1,1]
    
    for model in res_df['Model'].unique():
        model_data = res_df[res_df['Model'] == model]
        horizon_means = model_data.groupby('H')['RMSE'].mean()
        ax_horizon.plot(horizon_means.index, horizon_means.values, 
                       marker='o', label=model, linewidth=2.5, markersize=8)
    
    ax_horizon.set_xlabel('Horizonte (meses)')
    ax_horizon.set_ylabel('RMSE')
    ax_horizon.set_title('Evolución de RMSE por Horizonte', fontsize=14, pad=10)
    ax_horizon.legend(title='Modelo', loc='best')
    ax_horizon.grid(True, alpha=0.3)
    ax_horizon.set_xticks([1, 2, 3])
    
    plt.tight_layout()
    plt.savefig(COMP_DIR / 'metrics_comparison.png', dpi=150, bbox_inches='tight')
    plt.show()

# 2. Tabla resumen de mejores modelos
print("\n📋 TABLA RESUMEN - MEJORES MODELOS POR EXPERIMENTO:")
print("─" * 60)

best_models = res_df.groupby('Experiment').apply(
    lambda x: x.loc[x['RMSE'].idxmin()]
)[['Model', 'RMSE', 'MAE', 'R2']]

print(best_models.to_string())

# 3. Comparación con modelos originales si existen
old_metrics_path = BASE_PATH / 'models' / 'output' / 'Spatial_CONVRNN' / 'metrics_spatial.csv'
if old_metrics_path.exists():
    print("\n📊 COMPARACIÓN CON MODELOS ORIGINALES:")
    print("─" * 60)
    
    old_df = pd.read_csv(old_metrics_path)
    
    # Calcular mejoras promedio
    for exp in EXPERIMENTS.keys():
        print(f"\n{exp}:")
        
        # Mejores modelos nuevos
        new_best = res_df[res_df['Experiment'] == exp].groupby('Model')['RMSE'].mean().idxmin()
        new_rmse = res_df[(res_df['Experiment'] == exp) & (res_df['Model'] == new_best)]['RMSE'].mean()
        
        # Mejor modelo original
        old_best_rmse = old_df[old_df['Experiment'] == exp]['RMSE'].min()
        
        improvement = (old_best_rmse - new_rmse) / old_best_rmse * 100
        
        print(f"  • Mejor modelo nuevo: {new_best} (RMSE: {new_rmse:.4f})")
        print(f"  • Mejor RMSE original: {old_best_rmse:.4f}")
        print(f"  • Mejora: {improvement:.2f}%")

print("\n✅ Visualizaciones comparativas completadas!")
print(f"📂 Resultados guardados en: {COMP_DIR}")


In [ ]:
# ───────────────────────── ANÁLISIS DETALLADO DE RESULTADOS ─────────────────────────

if res_df is not None and len(res_df) > 0:
    print("\n" + "="*70)
    print("📊 ANÁLISIS DETALLADO DE RESULTADOS")
    print("="*70)
    
    # 1. Métricas por horizonte de predicción
    fig, axes = plt.subplots(1, 3, figsize=(20, 6))
    
    metrics = ['RMSE', 'MAE', 'R2']
    titles = ['RMSE por Horizonte', 'MAE por Horizonte', 'R² por Horizonte']
    colors = plt.cm.Set3(np.linspace(0, 1, len(res_df['Model'].unique())))
    
    for idx, (metric, title) in enumerate(zip(metrics, titles)):
        ax = axes[idx]
        
        # Obtener datos pivoteados
        data = res_df.groupby(['H', 'Model'])[metric].mean().unstack()
        
        # Plotear cada modelo
        for i, model in enumerate(data.columns):
            ax.plot(data.index, data[model], 
                   marker='o', 
                   label=model,
                   color=colors[i],
                   linewidth=2.5,
                   markersize=8,
                   markeredgewidth=2,
                   markeredgecolor='white')
        
        ax.set_xlabel('Horizonte (meses)', fontsize=12)
        ax.set_ylabel(metric, fontsize=12)
        ax.set_title(title, fontsize=14, fontweight='bold', pad=10)
        ax.grid(True, alpha=0.3, linestyle='--')
        ax.set_xticks(data.index)
        
        # Leyenda solo en el primer gráfico
        if idx == 0:
            ax.legend(title='Modelo', loc='best', frameon=True, fancybox=True, shadow=True)
    
    plt.tight_layout()
    plt.savefig(COMP_DIR / 'metrics_evolution_by_horizon.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    # 2. Tabla visual de métricas
    fig, ax = plt.subplots(figsize=(14, 8))
    ax.axis('tight')
    ax.axis('off')
    
    # Preparar datos para la tabla
    summary_data = []
    experiments = res_df['Experiment'].unique()
    models = res_df['Model'].unique()
    
    # Headers
    headers = ['Experimento', 'Modelo', 'RMSE↓', 'MAE↓', 'R²↑', 'Mejor H', 'Parámetros']
    
    for exp in experiments:
        for model in models:
            exp_model_data = res_df[(res_df['Experiment'] == exp) & (res_df['Model'] == model)]
            if not exp_model_data.empty:
                avg_rmse = exp_model_data['RMSE'].mean()
                avg_mae = exp_model_data['MAE'].mean()
                avg_r2 = exp_model_data['R2'].mean()
                best_h = exp_model_data.loc[exp_model_data['RMSE'].idxmin(), 'H']
                params = exp_model_data['Parameters'].iloc[0]
                
                summary_data.append([
                    exp, model, 
                    f'{avg_rmse:.4f}', 
                    f'{avg_mae:.4f}', 
                    f'{avg_r2:.4f}',
                    f'H={best_h}',
                    f'{params:,}'
                ])
    
    # Crear tabla
    table = ax.table(cellText=summary_data, colLabels=headers, 
                    cellLoc='center', loc='center')
    
    # Estilizar tabla
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1.2, 2)
    
    # Colorear celdas según rendimiento
    for i in range(len(summary_data)):
        # Obtener valores para comparación
        rmse_val = float(summary_data[i][2])
        mae_val = float(summary_data[i][3])
        r2_val = float(summary_data[i][4])
        
        # Encontrar min/max para normalización
        all_rmse = [float(row[2]) for row in summary_data]
        all_mae = [float(row[3]) for row in summary_data]
        all_r2 = [float(row[4]) for row in summary_data]
        
        # Normalizar y colorear RMSE (menor es mejor)
        rmse_norm = (rmse_val - min(all_rmse)) / (max(all_rmse) - min(all_rmse))
        rmse_color = plt.cm.RdYlGn(1 - rmse_norm)
        table[(i+1, 2)].set_facecolor(rmse_color)
        
        # Normalizar y colorear MAE (menor es mejor)
        mae_norm = (mae_val - min(all_mae)) / (max(all_mae) - min(all_mae))
        mae_color = plt.cm.RdYlGn(1 - mae_norm)
        table[(i+1, 3)].set_facecolor(mae_color)
        
        # Normalizar y colorear R² (mayor es mejor)
        r2_norm = (r2_val - min(all_r2)) / (max(all_r2) - min(all_r2))
        r2_color = plt.cm.RdYlGn(r2_norm)
        table[(i+1, 4)].set_facecolor(r2_color)
        
        # Colorear experimento
        exp_colors = {'BASIC': '#e8f4f8', 'KCE': '#f0e8f8', 'PAFC': '#f8e8f0'}
        table[(i+1, 0)].set_facecolor(exp_colors.get(summary_data[i][0], 'white'))
    
    # Colorear headers
    for j in range(len(headers)):
        table[(0, j)].set_facecolor('#4a86e8')
        table[(0, j)].set_text_props(weight='bold', color='white')
    
    plt.title('Resumen de Métricas por Modelo y Experimento\n(Verde=Mejor, Rojo=Peor)', 
             fontsize=16, fontweight='bold', pad=20)
    
    # Añadir leyenda
    plt.text(0.5, -0.05, '↓ = Menor es mejor, ↑ = Mayor es mejor', 
            transform=ax.transAxes, ha='center', fontsize=10, style='italic')
    
    plt.savefig(COMP_DIR / 'metrics_summary_table.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    # 3. Identificar el mejor modelo global
    print("\n🏆 MEJOR MODELO GLOBAL:")
    print("─" * 50)
    
    # Calcular score compuesto (normalizado)
    res_df['score'] = (
        (1 - (res_df['RMSE'] - res_df['RMSE'].min()) / (res_df['RMSE'].max() - res_df['RMSE'].min())) +
        (1 - (res_df['MAE'] - res_df['MAE'].min()) / (res_df['MAE'].max() - res_df['MAE'].min())) +
        ((res_df['R2'] - res_df['R2'].min()) / (res_df['R2'].max() - res_df['R2'].min()))
    ) / 3
    
    best_overall = res_df.loc[res_df['score'].idxmax()]
    print(f"Modelo: {best_overall['Model']}")
    print(f"Experimento: {best_overall['Experiment']}")
    print(f"Horizonte: {best_overall['H']}")
    print(f"RMSE: {best_overall['RMSE']:.4f}")
    print(f"MAE: {best_overall['MAE']:.4f}")
    print(f"R²: {best_overall['R2']:.4f}")
    print(f"Score compuesto: {best_overall['score']:.4f}")
    
    # 4. Análisis de mejora por horizonte
    print("\n📈 ANÁLISIS DE MEJORA POR HORIZONTE:")
    print("─" * 50)
    
    for h in [1, 2, 3]:
        h_data = res_df[res_df['H'] == h]
        best_h = h_data.loc[h_data['RMSE'].idxmin()]
        
        print(f"\nHorizonte {h}:")
        print(f"  • Mejor modelo: {best_h['Model']} - {best_h['Experiment']}")
        print(f"  • RMSE: {best_h['RMSE']:.4f}")
        print(f"  • R²: {best_h['R2']:.4f}")

print("\n✅ Análisis detallado completado!")


In [ ]:
# ───────────────────────── MOSTRAR PREDICCIONES MÁS RECIENTES ─────────────────────────
print("\n🖼️ PREDICCIONES MÁS RECIENTES:")
for exp in EXPERIMENTS.keys():
    exp_dir = OUT_ROOT / exp
    if exp_dir.exists():
        print(f"\n{exp}:")
        # Mostrar primera imagen de cada modelo
        for model in ADVANCED_MODELS.keys():
            img_path = exp_dir / f"{model}_1.png"
            gif_path = exp_dir / f"{model}.gif"
            
            if img_path.exists():
                from IPython.display import Image, display
                print(f"  {model} - Primera predicción (H=1):")
                display(Image(str(img_path), width=800))
                
            if gif_path.exists():
                print(f"  📹 GIF animado disponible: {gif_path}")

print("\n" + "="*70)
print("🎉 NOTEBOOK COMPLETADO!")
print("="*70)
print(f"\n📊 Resultados guardados en: {OUT_ROOT}")
print(f"📈 Métricas en: {OUT_ROOT/'metrics_advanced.csv'}")
print(f"🖼️ Visualizaciones en: {COMP_DIR}")
print("\n💡 Próximos pasos:")
print("   1. Revisar las métricas y seleccionar el mejor modelo")
print("   2. Hacer fine-tuning de hiperparámetros si es necesario")
print("   3. Entrenar un ensemble con los mejores modelos")
print("   4. Evaluar en datos más recientes o diferentes regiones")
